In [1]:
import org.apache.spark.sql.{DataFrame, Dataset, Row, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
val spark = SparkSession.builder.master("local[*]").appName("dataProcessing").getOrCreate()
import spark.implicits._

spark = org.apache.spark.sql.SparkSession@57b6885b


org.apache.spark.sql.SparkSession@57b6885b

In [9]:
import java.io.File
import scala.io.Source

val path = System.getProperty("user.dir")
val seasons = 2002 to 2015
val teams = 1 to 30

for (i <- seasons){
    for (j <- teams){
        val file = path + "/season_averages/" + i + "/" + j + ".json"
        try{
            var df_stats = spark.read.json(file)
            var elem = df_stats.select(explode($"data").as("data_flat"))
            var minutes = elem.select("data_flat.min")
            val split_min = elem.withColumn("ast", $"data_flat.ast")
            .withColumn("blk", $"data_flat.blk")
            .withColumn("dreb", $"data_flat.dreb")
            .withColumn("fg3_pct", $"data_flat.fg3_pct")
            .withColumn("fg3a", $"data_flat.fg3a")
            .withColumn("fg3m", $"data_flat.fg3m")
            .withColumn("fg_pct", $"data_flat.fg_pct")
            .withColumn("fga", $"data_flat.fga")
            .withColumn("fgm", $"data_flat.fgm")
            .withColumn("ft_pct", $"data_flat.ft_pct")
            .withColumn("fta", $"data_flat.fta")
            .withColumn("ftm", $"data_flat.ftm")
            .withColumn("games_played", $"data_flat.games_played")
            .withColumn("seconds", split($"data_flat.min", ":")(0) * 60 + split($"data_flat.min", ":")(1))
            .withColumn("oreb", $"data_flat.oreb")
            .withColumn("pf", $"data_flat.pf")
            .withColumn("player_id", $"data_flat.player_id")
            .withColumn("pts", $"data_flat.pts")
            .withColumn("reb", $"data_flat.reb")
            .withColumn("season", $"data_flat.season")
            .withColumn("stl", $"data_flat.stl")
            .withColumn("turnover", $"data_flat.turnover")
            .drop($"data_flat")
            
            val best_12 = split_min.orderBy(desc("seconds")).limit(12)
            val best_12_struct = best_12.withColumn("row", struct(col("ast"), col("blk"), col("dreb"), col("fg3_pct"), col("fg3a"), 
                                                                   col("fg3m"), col("fg_pct"), col("fga"), col("fgm"), 
                                                                   col("ft_pct"), col("fta"), col("ftm"), col("games_played"), 
                                                                   col("seconds"), col("oreb"), col("pf"), col("player_id"), 
                                                                   col("pts"), col("reb"), col("season"), col("stl"), col("turnover"))).agg(collect_list(col("row")).as("data"))
            
            val best_12_data = best_12_struct.select("data")
            best_12_data.write.mode("overwrite").format("json").save(path + "/season_averages/" + i + "/" + j)

            val best_12_36 = best_12.withColumn("ast", col("ast") / col("seconds") * 2160)
            .withColumn("blk", col("blk") / col("seconds") * 2160)
            .withColumn("dreb", col("dreb") / col("seconds") * 2160)
            .withColumn("fg3a", col("fg3a") / col("seconds") * 2160)
            .withColumn("fg3m", col("fg3m") / col("seconds") * 2160)
            .withColumn("fga", col("fga") / col("seconds") * 2160)
            .withColumn("fgm", col("fgm") / col("seconds") * 2160)
            .withColumn("fta", col("fta") / col("seconds") * 2160)
            .withColumn("ftm", col("ftm") / col("seconds") * 2160)
            .withColumn("oreb", col("oreb") / col("seconds") * 2160)
            .withColumn("pf", col("pf") / col("seconds") * 2160)
            .withColumn("pts", col("pts") / col("seconds") * 2160)
            .withColumn("reb", col("reb") / col("seconds") * 2160)
            .withColumn("seconds", col("seconds") / col("seconds") * 2160)
            .withColumn("stl", col("stl") / col("seconds") * 2160)
            .withColumn("turnover", col("turnover") / col("seconds") * 2160)
            
            val best_12_36_struct = best_12_36.withColumn("row", struct(col("ast"), col("blk"), col("dreb"), col("fg3_pct"), col("fg3a"), 
                                                                   col("fg3m"), col("fg_pct"), col("fga"), col("fgm"), 
                                                                   col("ft_pct"), col("fta"), col("ftm"), col("games_played"), 
                                                                   col("seconds"), col("oreb"), col("pf"), col("player_id"), 
                                                                   col("pts"), col("reb"), col("season"), col("stl"), col("turnover"))).agg(collect_list(col("row")).as("data"))
            
            val best_12_36_data = best_12_36_struct.select("data")
            best_12_36_data.write.mode("overwrite").format("json").save(path + "/season_averages/" + i + "/" + j + "-36")
            println("File " + file + " saved")
            
        }
        catch{
            case e: org.apache.spark.sql.AnalysisException => println("File " + file + " not found")
        }
    } 
}

File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2002/1.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2002/2.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2002/3.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2002/4.json not found
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2002/5.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2002/6.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2002/7.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2002/8.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2002/9.json sav

File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2004/13.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2004/14.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2004/15.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2004/16.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2004/17.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2004/18.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2004/19.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2004/20.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2004/21.jso

File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2006/25.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2006/26.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2006/27.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2006/28.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2006/29.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2006/30.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2006/31.json not found
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2007/1.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2007/2.j

File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2009/6.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2009/7.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2009/8.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2009/9.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2009/10.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2009/11.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2009/12.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2009/13.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2009/14.json sa

File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2011/18.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2011/19.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2011/20.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2011/21.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2011/22.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2011/23.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2011/24.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2011/25.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2011/26.jso

File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2013/30.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2013/31.json not found
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2014/1.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2014/2.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2014/3.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2014/4.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2014/5.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2014/6.json saved
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2014/7.json s

File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2016/10.json not found
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2016/11.json not found
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2016/12.json not found
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2016/13.json not found
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2016/14.json not found
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2016/15.json not found
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2016/16.json not found
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2016/17.json not found
File /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballG

path = /home/fernando/Desktop/DataIntensive-Project-Prediction-BasketballGames
seasons = Range(2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016)
teams = Range(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31)


lastException: Throwable = null


Range(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31)